In [1]:
from google.cloud import storage
import pandas as pd
import numpy as np

#### gs://marketplace-2xim6sjc/MedMNIST/120/artifactFiles/chestmnist_128.npz

# Initialize a client
client = storage.Client()

# Define the bucket and blob (file) names
bucket_name = 'marketplace-2xim6sjc'
blob_name = 'MedMNIST/120/artifactFiles/chestmnist_128.npz'

# Get the bucket and blob
bucket = client.bucket(bucket_name)
blob = bucket.blob(blob_name)

# Download the blob to a local file
local_file_path = 'chestmnist_128_local.npz'
blob.download_to_filename(local_file_path)

# Load the .npz file
data = np.load(local_file_path, allow_pickle=True)

# Access the data
for key in data:
    print(f"{key}: {data[key]}")

train_images: [[[200 127  47 ...   9   8   6]
  [172  90  25 ...  15   8   7]
  [164  85  22 ...  15   8   6]
  ...
  [190  94  23 ... 154 139 120]
  [196  97  28 ... 173 162 145]
  [202 103  37 ... 193 184 169]]

 [[  3   3   3 ...   1   5  23]
  [  3   3   3 ...   1   5  23]
  [  3   3   3 ...   1   5  22]
  ...
  [  2   2   2 ...   1   3  14]
  [  3   3   3 ...   2   6  20]
  [  7   7   7 ...   6  12  32]]

 [[255 253 251 ... 242 245 241]
  [248 243 236 ... 232 236 233]
  [230 205 179 ... 215 220 219]
  ...
  [ 41  30  28 ...  28  28  30]
  [ 41  30  28 ...  28  28  30]
  [ 41  29  28 ...  28  28  30]]

 ...

 [[  0   0   0 ...   0   0   0]
  [  0   0   0 ...   0   0   0]
  [  0   0   0 ...   0   0   0]
  ...
  [  0   0   0 ...   0   0   0]
  [  0   0   0 ...   0   0   0]
  [  0   0   0 ...   0   0   0]]

 [[ 25   6   0 ...  53  55  57]
  [ 22   3   0 ...  11  12  13]
  [ 20   3   0 ...   0   0   0]
  ...
  [ 42  52  66 ...   0   0   0]
  [ 41  52  66 ...   0   0   0]
  [ 43  55  69

In [4]:
from google.cloud import storage
import numpy as np
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

In [2]:
!pip install google-cloud-storage numpy torch torchvision matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 18.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 32.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 11.0 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 21.3 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 977.7 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 31.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 62.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 78.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 55.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 62.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 33.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━

In [5]:
# Extract the data
x_train = data['train_images']
y_train = data['train_labels']
x_val = data['val_images']
y_val = data['val_labels']
x_test = data['test_images']
y_test = data['test_labels']

# Normalize the images
x_train = x_train / 255.0
x_val = x_val / 255.0
x_test = x_test / 255.0

# Convert to PyTorch tensors
x_train = torch.tensor(x_train, dtype=torch.float32).unsqueeze(1)  # Add channel dimension
y_train = torch.tensor(y_train, dtype=torch.long)
x_val = torch.tensor(x_val, dtype=torch.float32).unsqueeze(1)  # Add channel dimension
y_val = torch.tensor(y_val, dtype=torch.long)
x_test = torch.tensor(x_test, dtype=torch.float32).unsqueeze(1)  # Add channel dimension
y_test = torch.tensor(y_test, dtype=torch.long)

# Create DataLoader
train_dataset = TensorDataset(x_train, y_train)
val_dataset = TensorDataset(x_val, y_val)
test_dataset = TensorDataset(x_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [18]:
len(y_train)

78468

In [22]:
len(x_train)

78468

In [23]:
len(x_val)

11219

In [24]:
len(x_test)

22433

In [19]:
len(y_val)

11219

In [20]:
len(y_test)

22433

In [28]:
# Extract the data
x_train = data['train_images']
y_train = data['train_labels']
x_val = data['val_images']
y_val = data['val_labels']
x_test = data['test_images']
y_test = data['test_labels']

# Normalize the images
x_train = x_train / 255.0
x_val = x_val / 255.0
x_test = x_test / 255.0

# Convert to PyTorch tensors
x_train = torch.tensor(x_train, dtype=torch.float32).unsqueeze(1)  # Add channel dimension
y_train = torch.tensor(y_train, dtype=torch.float32)  # Convert to float for BCEWithLogitsLoss
x_val = torch.tensor(x_val, dtype=torch.float32).unsqueeze(1)  # Add channel dimension
y_val = torch.tensor(y_val, dtype=torch.float32)  # Convert to float for BCEWithLogitsLoss
x_test = torch.tensor(x_test, dtype=torch.float32).unsqueeze(1)  # Add channel dimension
y_test = torch.tensor(y_test, dtype=torch.float32)  # Convert to float for BCEWithLogitsLoss

# Create DataLoader
train_dataset = TensorDataset(x_train, y_train)
val_dataset = TensorDataset(x_val, y_val)
test_dataset = TensorDataset(x_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Define the CNN Model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 32 * 32, 128)  # Update this line based on the input size
        self.fc2 = nn.Linear(128, 14)  # Assuming 14 classes for ChestMNIST

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)  # Flatten the tensor
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = CNN()
criterion = nn.BCEWithLogitsLoss()  # Use BCEWithLogitsLoss for multi-label classification
optimizer = optim.Adam(model.parameters(), lr=0.001)

Epoch 1/2, Training Loss: 0.17746433135374057, Validation Loss: 0.17188468816774813
Epoch 2/2, Training Loss: 0.17123841004295862, Validation Loss: 0.1683678675696931
Accuracy of the model on the test images: 1326.4030669103554%


In [36]:

# Define the Improved CNN Model
class ImprovedCNN(nn.Module):
    def __init__(self):
        super(ImprovedCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout = nn.Dropout(0.5)
        self.fc1 = nn.Linear(128 * 16 * 16, 256)  # Update this line based on the input size
        self.fc2 = nn.Linear(256, 14)  # Assuming 14 classes for ChestMNIST

    def forward(self, x):
        x = self.pool(torch.relu(self.bn1(self.conv1(x))))
        x = self.pool(torch.relu(self.bn2(self.conv2(x))))
        x = self.pool(torch.relu(self.bn3(self.conv3(x))))
        x = x.view(x.size(0), -1)  # Flatten the tensor
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

model = ImprovedCNN()
criterion = nn.BCEWithLogitsLoss()  # Use BCEWithLogitsLoss for multi-label classification
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [37]:
# Train the Model
def train_model(model, train_loader, val_loader, criterion, optimizer, epochs=2):
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        val_loss = 0.0
        model.eval()
        with torch.no_grad():
            for images, labels in val_loader:
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

        print(f"Epoch {epoch+1}/{epochs}, Training Loss: {running_loss/len(train_loader)}, Validation Loss: {val_loss/len(val_loader)}")

In [ ]:
train_model(model, train_loader, val_loader, criterion, optimizer, epochs=5)

Epoch 1/5, Training Loss: 0.18346074380788166, Validation Loss: 0.17133278653702952
Epoch 2/5, Training Loss: 0.17843918819602855, Validation Loss: 0.18057209947569805
Epoch 3/5, Training Loss: 0.1751040847396695, Validation Loss: 0.16863458417356014


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Evaluate the Model
def evaluate_model(model, test_loader):
    model.eval()
    all_labels = []
    all_predictions = []
    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images)
            predicted = (outputs > 0.5).float()  # Convert logits to binary predictions
            all_labels.append(labels.cpu().numpy())
            all_predictions.append(predicted.cpu().numpy())

    all_labels = np.concatenate(all_labels, axis=0)
    all_predictions = np.concatenate(all_predictions, axis=0)

    accuracy = accuracy_score(all_labels, all_predictions)
    precision = precision_score(all_labels, all_predictions, average='macro')
    recall = recall_score(all_labels, all_predictions, average='macro')
    f1 = f1_score(all_labels, all_predictions, average='macro')

    print(f"Accuracy: {accuracy * 100:.2f}%")
    print(f"Precision: {precision * 100:.2f}%")
    print(f"Recall: {recall * 100:.2f}%")
    print(f"F1 Score: {f1 * 100:.2f}%")

evaluate_model(model, test_loader)